This is the beginning of the notebook.

##ADM4142-A Fundamentals of Data science <br>
The goal of this notebook is to retrieve and stage the source datasets into the format used in the dimensional model for analysis.

This notebook generates the Economy_dimension of the weather/tourism/economy data frame.

In [ ]:
import pandas as pd


In [ ]:
# Define the URL for the dataset
dataset_url = 'https://raw.githubusercontent.com/noobstang/cscsi4142-project-datasets/master/data/24100043_monthlyProvincial_tourists.csv'

# Load the dataset
tourism_data = pd.read_csv(dataset_url)

# Remove apostrophes from all cells
tourism_data = tourism_data.replace({"\"": ""}, regex=True)

#tourism_data.head(10)

In [ ]:
#tourism_data.head()
tourism_data.tail()

Now that we've imported the dataset, we need to transform it into the format that includes only the original columns.

In [ ]:

# Section 4: Filter rows by date
start_date = '1990-01'
end_date = '2023-12'
tourism_data['REF_DATE'] = pd.to_datetime(tourism_data['REF_DATE'])
tourism_data = tourism_data[(tourism_data['REF_DATE'] >= start_date) & (tourism_data['REF_DATE'] <= end_date)]

# Section 5: Filter rows by GEO column
valid_geo = ['Canada', 'Alberta', 'British Columbia', 'Ontario', 'Quebec']
tourism_data = tourism_data[tourism_data['GEO'].isin(valid_geo)]
#tourism_data = tourism_data[tourism_data['Seasonal adjustment'] == 'Unadjusted']

tourism_data.head()

In [ ]:
# Section 6: Prepare columns for keys (to be mapped later)
tourism_data['Date_key'] = ""
tourism_data['Weather_key'] = ""
tourism_data['Location_key'] = ""
tourism_data['Economy_key'] = ""

# Section 7: Adjust columns based on Traveller characteristics
tourism_data['Total non-resident tourists'] = tourism_data.loc[tourism_data['Traveller characteristics'] == 'Total non resident tourists', 'VALUE']
tourism_data['United states tourists'] = tourism_data.loc[tourism_data['Traveller characteristics'] == 'United States tourists', 'VALUE']
tourism_data['Non-US foreign tourists'] = tourism_data.loc[tourism_data['Traveller characteristics'] == 'Tourists from countries other than United States', 'VALUE']
tourism_data['Canadian tourists returning from U.S.'] = tourism_data.loc[tourism_data['Traveller characteristics'] == 'Canadian tourists returning from United States', 'VALUE']
tourism_data['Canadian tourists returning from abroad'] = tourism_data.loc[tourism_data['Traveller characteristics'] == 'Canadian tourists returning from countries other than United States', 'VALUE']
tourism_data['Seasonally adjusted'] = tourism_data['Seasonal adjustment']



# Section 9: Merge entries based on matching GEO and REF_DATE values
tourism_data = tourism_data.groupby(['REF_DATE', 'GEO', 'Seasonal adjustment'], as_index=False).agg({
    'Total non-resident tourists': 'sum',
    'United states tourists': 'sum',
    'Non-US foreign tourists': 'sum',
    'Canadian tourists returning from U.S.': 'sum',
    'Canadian tourists returning from abroad': 'sum',
    #'Seasonally adjusted': 'first',  # Since it's unadjusted, all values should be the same
})

# Section 8: Select entries with "Seasonal adjustment" value "Unadjusted"
unadjusted_data = tourism_data[tourism_data['Seasonal adjustment'] == 'Unadjusted']

## to be used later
adjusted_data = tourism_data.copy()
adjusted_data = adjusted_data[tourism_data['Seasonal adjustment'] == 'Seasonally adjusted']
##

# make a copy to perform manipulations
merged_data = unadjusted_data

# Section 10: Calculate "Total non-resident tourists"
merged_data['Total non-resident tourists'] = merged_data['United states tourists'] + merged_data['Non-US foreign tourists']

# Section 10: Fill missing values for other attributes
merged_data['Seasonal adjustment'].fillna("", inplace=True)
merged_data['Date_key'] = ""
merged_data['Weather_key'] = ""
merged_data['Location_key'] = ""
merged_data['Economy_key'] = ""


In [ ]:
merged_data.head(20)

Now, add the date dimension

In [ ]:
# Section 12: Load date dimension data
date_dimension_url = "https://raw.githubusercontent.com/noobstang/cscsi4142-project-datasets/master/dimension/date.csv"
date_dimension = pd.read_csv(date_dimension_url)
date_dimension['date_iso'] = pd.to_datetime(date_dimension['date_iso'])

# Section 13: Merge with date dimension based on REF_DATE and date_iso
merged_data = pd.merge(merged_data, date_dimension, left_on='REF_DATE', right_on='date_iso', how='left')

# Section 14: Fill Date_key column with mapped values
merged_data['Date_key'] = merged_data['Date_key_y']

# Section 15: Drop unnecessary columns
merged_data.drop(columns=['Date_key_x', 'Date_key_y', 'date_iso', 'day_of_week'], inplace=True)

In [ ]:
# Count the number of missing values in the Date_key column
missing_date_keys = merged_data['Date_key'].isna().sum()
print("Number of missing values in the Date_key column:", missing_date_keys)


Now, add the location dimension

In [ ]:
# Section 17: Load location dimension data
location_dimension_url = "https://raw.githubusercontent.com/noobstang/cscsi4142-project-datasets/master/dimension/location.csv"
location_dimension = pd.read_csv(location_dimension_url)
#location_dimension = pd.merge(location_dimension, pd dimension)
# Merge with location dimension based on Date_key
location_dimension = pd.merge(location_dimension, date_dimension[['Date_key', 'year']], on='Date_key', how='left')
#location_dimension['Date_key']

# Section 24: Fill Economy_key column with mapped values
#location_dimension['Date_key'] = location_dimension['Date_key_y']

location_dimension.head()

In [ ]:
# Section 18: Merge with location dimension based on Date_key and GEO
merged_data = pd.merge(merged_data, location_dimension, left_on=['year', 'GEO'], right_on=['year', 'location'], how='left')

# Section 19: Fill Location_key column with mapped values
merged_data['Location_key'] = merged_data['Location_key_y']
merged_data['Date_key'] = merged_data['Date_key_y']

# Section 20: Drop unnecessary columns
merged_data.drop(columns=['Location_key_x', 'Location_key_y', 'Date_key_x', 'Date_key_y', 'location', 'population'], inplace=True)

# Section 21: Save the merged data as "tourism_fact_table.csv"
#merged_data.to_csv("tourism_fact_table.csv", index=False)


In [ ]:
# Count the number of missing values in the Location_key column
missing_location_keys = merged_data['Location_key'].isna().sum()
print("Number of missing values in the Location_key column:", missing_location_keys)


In [ ]:
# Check the 10 entries starting from the 200th entry
#print(merged_data.iloc[100:200])

merged_data.head()

Now, add the economy dimension

In [ ]:
# Section 22: Load economy dimension data
economy_dimension_url = "https://raw.githubusercontent.com/noobstang/cscsi4142-project-datasets/master/dimension/economy.csv"
#economy_dimension = pd.read_csv(economy_dimension_url)
economy_dimension = pd.read_csv(economy_dimension_url, usecols=['Date_key', 'Location_key', 'Economy_key'])
# Merge with date dimension based on Date_key to get year
economy_dimension = pd.merge(economy_dimension, date_dimension[['Date_key', 'year']], on='Date_key', how='left')

In [ ]:
# Section 23: Merge with economy dimension based on Date_key and Location_key
temp_merged_data = pd.merge(merged_data, economy_dimension, on=['year', 'Location_key'], how='left')

# Section 19: Fill Location_key column with mapped values
merged_data['Economy_key'] = temp_merged_data['Economy_key_y']
#merged_data['Date_key'] = merged_data['Date_key_y']

# Section 24: Fill Economy_key column with mapped values
#merged_data['Economy_key'] = temp_merged_data['Economy_key_y']

# Section 25: Drop unnecessary columns
#merged_data.drop(columns=['Economy_key_x', 'Economy_key_y', 'Date_key_x', 'Date_key_y'], inplace=True)

# Section 26: Save the merged data as "tourism_fact_table.csv"
#merged_data.to_csv("tourism_fact_table.csv", index=False)


In [ ]:
merged_data.tail(10)

In [ ]:
# Count the number of missing values in the Location_key column
missing_economy_keys = merged_data['Economy_key'].isna().sum()
print("Number of missing values in the Economy_key column:", missing_economy_keys)


Finally, add the keys from Weather dimension. There are multiple Weather_keys corresponding to different cities from the same province when merging using Location_key and Date_key. These are combined into a single cell.


In [ ]:
import json

# Step 1: Load the weather dimension data
weather_dimension_url = "https://raw.githubusercontent.com/noobstang/cscsi4142-project-datasets/master/dimension/weather_final.csv"
weather_dimension = pd.read_csv(weather_dimension_url, usecols=['Date_key', 'Location_key', 'Weather_key'])

# Step 2: Preprocess the Weather Data
# Aggregate Weather_key for each combination of Location_key and Date_key into a list, then convert to a JSON string
weather_aggregated = weather_dimension.groupby(['Date_key', 'Location_key']).agg(
    Weather_keys=('Weather_key', lambda x: json.dumps(list(x)))  # Convert the list of Weather_keys to JSON string
).reset_index()

In [ ]:
weather_aggregated.head()

In [ ]:
# Step 3: Merge with the Main Data
merged_data_with_weather = pd.merge(merged_data, weather_aggregated, on=['Location_key', 'Date_key'], how='left')
merged_data['Weather_key'] = merged_data_with_weather['Weather_keys']

# If needed, replace NaN values in Weather_keys with a default value, e.g., an empty list as a JSON string
#merged_data['Weather_key'].fillna(json.dumps([]), inplace=True)


In [ ]:
merged_data.tail()

In [ ]:
# Count the number of missing values in the Location_key column
missing_weather_keys = merged_data['Weather_key'].isna().sum()
print("Number of missing values in the Weather_key column:", missing_weather_keys)


Completed. Note that There are no associated weather keys for rows whose location is "Canada", since there is no associated weather data included for the entire country.

Now we need to perform the same procedure for the seasonally adjusted data, and concatenate the two sets into one final dataset.

In [ ]:
#merged_data_with_weather.head()
final_data_unadjusted = merged_data.copy()

In [ ]:
# make a copy to perform manipulations
merged_data = adjusted_data

# Section 10: Calculate "Total non-resident tourists"
merged_data['Total non-resident tourists'] = merged_data['United states tourists'] + merged_data['Non-US foreign tourists']

# Section 10: Fill missing values for other attributes
merged_data['Seasonal adjustment'].fillna("", inplace=True)
merged_data['Date_key'] = ""
merged_data['Weather_key'] = ""
merged_data['Location_key'] = ""
merged_data['Economy_key'] = ""

### Date dimension
# Section 12: Load date dimension data

# Section 13: Merge with date dimension based on REF_DATE and date_iso
merged_data = pd.merge(merged_data, date_dimension, left_on='REF_DATE', right_on='date_iso', how='left')

# Section 14: Fill Date_key column with mapped values
merged_data['Date_key'] = merged_data['Date_key_y']

# Section 15: Drop unnecessary columns
merged_data.drop(columns=['Date_key_x', 'Date_key_y', 'date_iso', 'day_of_week'], inplace=True)

### Location
# Section 18: Merge with location dimension based on Date_key and GEO
merged_data = pd.merge(merged_data, location_dimension, left_on=['year', 'GEO'], right_on=['year', 'location'], how='left')

# Section 19: Fill Location_key column with mapped values
merged_data['Location_key'] = merged_data['Location_key_y']
merged_data['Date_key'] = merged_data['Date_key_y']

# Section 20: Drop unnecessary columns
merged_data.drop(columns=['Location_key_x', 'Location_key_y', 'Date_key_x', 'Date_key_y', 'location', 'population'], inplace=True)

### Economy dimension
merged_data['Location_key'] = merged_data['Location_key'].astype('int64')

# Section 23: Merge with economy dimension based on Date_key and Location_key
temp_merged_data_2 = pd.merge(merged_data, economy_dimension, on=['year', 'Location_key'], how='left')

# Section 19: Fill Location_key column with mapped values
merged_data['Economy_key'] = temp_merged_data_2['Economy_key_y']

### Weather dimension
# Step 3: Merge with the Main Data
merged_data_with_weather_2 = pd.merge(merged_data, weather_aggregated, on=['Location_key', 'Date_key'], how='left')
merged_data['Weather_key'] = merged_data_with_weather_2['Weather_keys']

# If needed, replace NaN values in Weather_keys with a default value, e.g., an empty list as a JSON string
#merged_data['Weather_key'].fillna(json.dumps([]), inplace=True)


In [ ]:
merged_data.head()

In [ ]:
final_data_adjusted = merged_data

combined_data = pd.concat([final_data_unadjusted, final_data_adjusted], ignore_index=True)

In [ ]:
combined_data.tail(10)

Combine, sort, and organize attributes and rows

In [ ]:
import numpy as np

# Final selection and sorting of the columns
final_data = combined_data[['Date_key', 'Weather_key', 'Location_key', 'Economy_key',
                                          'Total non-resident tourists', 'United states tourists',
                                          'Non-US foreign tourists', 'Canadian tourists returning from U.S.',
                                          'Canadian tourists returning from abroad', 'Seasonal adjustment']].copy()

final_data['Seasonal adjustment'] = final_data['Seasonal adjustment'].map({"Unadjusted": False, "Seasonally adjusted": True})
#final_data.rename(columns={'Weather_key': 'Weather_keys'}, inplace=True)
final_data.rename(columns={'Seasonal adjustment': 'Seasonally adjusted'}, inplace=True)

# Sort by Date_key (and any other keys as secondary sorts if you wish)
final_data = final_data.sort_values(by=['Date_key', 'Location_key'])

# Convert 'Economy_key' to numeric, coercing errors, then to int64 where possible
#final_data['Weather_key'] = pd.to_numeric(final_data['Weather_key'], errors='ignore').astype('Int64')
final_data['Economy_key'] = pd.to_numeric(final_data['Economy_key'], errors='ignore').astype('Int64')
# Replace NaN values in "Weather_key" column with pd.NA
#final_data['Weather_key'] = final_data['Weather_key'].replace(np.nan, pd.NA)

# Assuming df is your DataFrame and it has a column named "Weather_key"
# First, ensure the column is of a type that supports pd.NA. If it's not, this step can be omitted.
final_data['Economy_key'] = final_data['Economy_key'].astype('object')

# Replace pd.NA with np.nan
final_data['Economy_key'] = final_data['Economy_key'].fillna(np.nan)


# Reset the index after sorting
final_data.reset_index(drop=True, inplace=True)

# Display the first few rows of the final sorted dataset
#print(final_data.head())

View final data

In [ ]:
final_data.head(20)

In [ ]:
final_data.tail(20)

In [ ]:
# Save to file
#final_data.to_csv("tourism_fact_table.csv", index=False)